In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import shutil
import os
import pandas as pd
import cv2
import pydicom
import numpy as np

In [ ]:
# This file contains the image names and their class labels

df1 = pd.read_csv('D://YOLO-Vidya//stage_2_detailed_class_info.csv')
df1

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [ ]:
# Taking the row position of the images which belong to the class No Lung Opacity / Not Normal
index= df1[df1['class']=='No Lung Opacity / Not Normal'].index

In [ ]:
#dropping the third class No Lung Opacity / Not Normal
df1.drop(index,inplace=True)

In [ ]:
df1

,patientId,class
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
5,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
8,00704310-78a8-4b38-8475-49f4573b2dbb,Lung Opacity
9,00704310-78a8-4b38-8475-49f4573b2dbb,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [ ]:
df1['class'].value_counts()

Lung Opacity    9555
Normal          8851
Name: class, dtype: int64

In [ ]:
#Getting Image names of the 2 classes. There will be duplicates as there are multiple bounding boxes for single images.
# Hence removing the duplicates
PatientID = list(set(df1['patientId'].values))

In [ ]:
PatientID[0:5]

['5039abb8-29f6-4e8a-92e6-fbcb74804a7e',
 '5eb5e5a8-076b-4bf8-ae80-26f6334e8790',
 '5b9fb157-f24d-4d07-abbc-bdf3356e1b29',
 '9e6239f2-9352-4bad-a19a-22538ddd0e27',
 '5a127388-ca5d-45a3-b0b5-1f925270d98c']

In [ ]:
# List of images with all three classes in training set
train_images_old = os.listdir('D://YOLO-Vidya//train')

In [ ]:
train_images_old[0:5]

['0000a175-0e68-4ca4-b1af-167204a7e0bc.dcm',
 '0005d3cc-3c3f-40b9-93c3-46231c3eb813.dcm',
 '000686d7-f4fc-448d-97a0-44fa9c5d3aa6.dcm',
 '000e3a7d-c0ca-4349-bb26-5af2d8993c3d.dcm',
 '00100a24-854d-423d-a092-edcf6179e061.dcm']

In [ ]:
src_train = "D://YOLO-Vidya//train"
dest_train = "D://YOLO-Vidya//train_new"

# Move only the 2 classes from original training directory to another directory
for img_name in PatientID:
    img_name = img_name+'.dcm'
    if img_name in train_images_old:
        shutil.move(src_train+"//"+img_name,dest_train)
    else:
        print(f'Image not found - {img_name}')

In [ ]:
#Convert all .dcm images to .jpg/.png images

PNG = False
# Specify the .dcm folder path
folder_path = "D://YOLO-Vidya//train_new"
# Specify the output jpg/png folder path
jpg_folder_path = "D://YOLO-Vidya//converted_train_images"
images_path = os.listdir(folder_path)
for n, image in enumerate(images_path):
    ds = pydicom.dcmread(os.path.join(folder_path, image))
    pixel_array_numpy = ds.pixel_array
    if PNG == False:
        image = image.replace('.dcm', '.jpg')
    else:
        image = image.replace('.dcm', '.png')
    cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)

In [ ]:
# Now from the images directory containing two classes, identify the Normal and Pneumonia and move them into separate folders

def move_images_as_per_label(path,images):
    for image in images:
        img = image[:-4]
        classes = list(set(df1[df1['patientId']==img]['class']))
        if classes[0] == 'Lung Opacity':
            shutil.move(path+"//"+image,path+"//Pneumonia")
        else:
            shutil.move(path+"//"+image,path+"//Normal")

In [ ]:
img_path = "D://YOLO-Vidya//All_jpg_images"
images = os.listdir("D://YOLO-Vidya//All_jpg_images")

In [ ]:
move_images_as_per_label(img_path,images)

In [ ]:
Pneumonia_images = os.listdir(img_path+"//Pneumonia")
Normal_images = os.listdir(img_path+"//Normal")

train_path = "D://YOLO-Vidya//train"
valid_path = "D://YOLO-Vidya//valid"

In [ ]:
# do a 80/20 split of these Pneumonia images and normal images
# Move 80 percent of Pnuemonia and Normal images into training folder
# Move 20 percent of images into validation folder

train_set = Pneumonia_images[:round(len(Pneumonia_images)*0.8)]
train_set.extend(Normal_images[:round(len(Normal_images)*0.8)])

for image in train_set:
    if os.path.exists(img_path+"//Pneumonia//"+image):
        shutil.move(img_path+"//Pneumonia//"+image,train_path)
    elif os.path.exists(img_path+"//Normal//"+image):
        shutil.move(img_path+"//Normal//"+image,train_path)

In [ ]:
Pneumonia_images = os.listdir(img_path+"//Pneumonia")
Normal_images = os.listdir(img_path+"//Normal")

len(Normal_images)

1770

In [ ]:
for img in Pneumonia_images:
    shutil.move(img_path+"//Pneumonia//"+img,valid_path)
    
for img in Normal_images:
    shutil.move(img_path+"//Normal//"+img,valid_path)

In [ ]:
"""To train the Yolo-v5 model, we need to organize our dataset structure and it requires images (.jpg/.png, etc.,) 
and it’s corresponding labels in .txt format.

STRUCTURE OF .txt FILE :
- One row per object.
- Each row is class x_center y_center width height format.
- Box coordinates must be in normalized xywh format (from 0–1). If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
- Class numbers are zero-indexed (start from 0)

All our images are of fixed width and height. And I am just maintaining the same size

In case of different sizes. Images and their coordinates have to be adjusted properly

"""

img_width = 1024
img_height = 1024

def x_center(df2):
    if df2['width']==0:
        return 0
    else:
        return int(df2['x'] + (df2['width']/2))
def y_center(df2):
    if df2['height']==0:
        return 0
    else:
        return int(df2['y'] + (df2['height']/2))
def w_norm(df2):
    if df2['width']==0:
        return 0
    else:
        return df2/img_width
def h_norm(df):
    if df2['height']==0:
        return 0
    else:
        return df2/img_height

df2 = pd.read_csv("D://YOLO-Vidya//stage_2_train_labels.csv")

df2 = df2.astype({"x":'int', "y":'int',"width":'int', "height":'int'}) 

df2['x_center'] = df2.apply(x_center, axis=1)
df2['y_center'] = df2.apply(y_center, axis=1)

df2['x_center_norm'] = df2['x_center'].apply(w_norm)
df2['width_norm'] = df2['width'].apply(w_norm)

df2['y_center_norm'] = df2['y_center'].apply(h_norm)
df2['height_norm'] = df2['height'].apply(h_norm)

In [ ]:
df2.head(30)

,patientId,x,y,width,height,Target,x_center,y_center,x_center_norm,width_norm,y_center_norm,height_norm
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,0,0,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,0,0,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000
2,00322d4d-1c29-4943-afc9-b6754be640eb,0,0,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,0,0,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000
4,00436515-870c-4b36-a041-de91049b9ab4,264,152,213,379,1,370,341,0.22981,0.20481,0.25577,0.36442
5,00436515-870c-4b36-a041-de91049b9ab4,562,152,256,453,1,690,378,0.39327,0.24615,0.29135,0.43558
6,00569f44-917d-4c86-a842-81832af98c30,0,0,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,0,0,0,0,0,0,0,0.00000,0.00000,0.00000,0.00000
8,00704310-78a8-4b38-8475-49f4573b2dbb,323,577,160,104,1,403,629,0.23269,0.15385,0.32788,0.10000
9,00704310-78a8-4b38-8475-49f4573b2dbb,695,575,162,137,1,776,643,0.41250,0.15577,0.34231,0.13173


In [ ]:
df2 = pd.read_csv("D://YOLO-Vidya//stage_2_train_labels.csv")

train_images = os.listdir("D://YOLO-Vidya//images//train")
valid_images = os.listdir("D://YOLO-Vidya//images//valid")

label_path = "D:\\YOLO-Vidya\\labels"

In [ ]:
# Preparing a .txt file for each image with labels and coordinates of the same

for image in train_images:
    yolo_list = []
    img = image[:-4]
    for _,row in df2[df2['patientId'] == img].iterrows():
        yolo_list.append([row.Target, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = label_path+"\\train\\"+img+".txt"
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])


In [ ]:
for image in valid_images:
    yolo_list = []
    img = image[:-4]
    for _,row in df2[df2['patientId'] == img].iterrows():
        yolo_list.append([row.Target, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = label_path+"\\valid\\"+img+".txt"
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])


In [ ]:
print(len(train_images))
print(len(valid_images))

print(len(os.listdir("D:\\YOLO-Vidya\\labels\\train")))
print(len(os.listdir("D:\\YOLO-Vidya\\labels\\valid")))

11891
2972
11891
2972


In [ ]:
### END OF DATA PREPARATION ###